In [ ]:
! pip install --upgrade scikit-learn

In [ ]:
import pandas
train = pandas.read_csv('https://github.com/Ashuto7h/ML-works/blob/main/HE%20%20challenges/reduce%20market%20waste%20-%20HE%20april%202021/train.csv?raw=true', na_values='?')
valid = pandas.read_csv('https://github.com/Ashuto7h/ML-works/blob/main/HE%20%20challenges/reduce%20market%20waste%20-%20HE%20april%202021/test.csv?raw=true',na_values='?')
train

# Profiling Dataset

In [ ]:
# !pip install --upgrade pandas_profiling 

In [ ]:
# from pandas_profiling import ProfileReport
# train.profile_report()

# Analysis and Preprocessing

### 23. Success_probability

In [ ]:
import matplotlib.pyplot as pyplot
import seaborn
train.loc[(train.Success_probability > 100) | (train.Success_probability < 0), 'Success_probability'] = train['Success_probability'].median()
fig = pyplot.figure(figsize = (25,5))
pyplot.subplot(121)
seaborn.histplot(data =train,x = train['Success_probability'])

### 1. Deal_title 

In [ ]:
deal_title = valid['Deal_title']
train.drop(columns = 'Deal_title', inplace= True)
valid.drop(columns = 'Deal_title', inplace= True)

### 2. Lead_name
### 8. Contact_no
### 13. POC_name
### 15. Lead_POC_email

In [ ]:
cols = ['Lead_name', 'Contact_no','POC_name','Lead_POC_email']
train = train.drop(columns = cols)
valid= valid.drop(columns = cols)

### 3. Industry
not affect much

In [ ]:
print(train['Industry'].value_counts())
train['Industry'].fillna(value = 'Banks', inplace =True)
valid['Industry'].fillna(value = 'Banks', inplace =True)

### 4. Deal_value
### 5. Weighted Amount

In [ ]:
import numpy
cols = ['Deal_value', 'Weighted_amount']
for col in cols:
    train[col] = train[col].str.replace(r'\D', '')
    valid[col] = valid[col].str.replace(r'\D', '')
    train[col].fillna(value = -1, inplace = True)
    valid[col].fillna(value = -1, inplace = True)
    train[col] = train[col].astype(int)
    valid[col] = valid[col].astype(int)


In [ ]:
pyplot.figure(figsize = (25,10))
i = 1
for col in cols:
    pyplot.subplot(int(str(1)+ str(len(cols)) + str(i)))
    seaborn.boxplot(data=train, y = col)
    i += 1

In [ ]:
print(train['Deal_value'].isna().sum())
train['Deal_value'].replace(-1, train['Deal_value'].mean(),inplace = True)
valid['Deal_value'].replace(-1, train['Deal_value'].mean(),inplace = True)
train['Weighted_amount'].replace(-1, train['Weighted_amount'].median(),inplace = True)
valid['Weighted_amount'].replace(-1, train['Weighted_amount'].median(),inplace = True)


### 6. Date_of_creation

In [ ]:
train['Date_of_creation'] = pandas.to_datetime(train['Date_of_creation'], format = '%Y-%m-%d',errors = 'coerce')
valid['Date_of_creation'] = pandas.to_datetime(valid['Date_of_creation'], format = '%Y-%m-%d',errors = 'coerce')

train['creation_year'] = train['Date_of_creation'].dt.year
train['creation_week'] = train['Date_of_creation'].dt.isocalendar().week
train['creation_day'] = train['Date_of_creation'].dt.day

valid['creation_year'] = valid['Date_of_creation'].dt.year
valid['creation_week'] = valid['Date_of_creation'].dt.isocalendar().week
valid['creation_day'] = valid['Date_of_creation'].dt.day

train.drop(columns=['Date_of_creation'],inplace = True)
valid.drop(columns=['Date_of_creation'],inplace = True)

### 11. Geography

In [ ]:
train['Geography'].fillna(value = 'USA', inplace =True)
valid['Geography'].fillna(value = 'USA', inplace =True)

### 12. Location

In [ ]:
vc = train['Location'].value_counts()
train['Location'].fillna(value = vc.index[0], inplace = True)
valid['Location'].fillna(value = vc.index[0], inplace = True)


### 19. Last_lead_update
affects

In [ ]:
vc = train['Last_lead_update'].value_counts()
train['Last_lead_update'].fillna(value = vc.index[0], inplace =True)
valid['Last_lead_update'].fillna(value = vc.index[0], inplace =True)

### 21.  Resource

In [ ]:
train['Resource'].fillna(value = 'No', inplace = True)
valid['Resource'].fillna(value = 'No', inplace = True)

# Visualization

In [ ]:
train.describe()

In [ ]:
pyplot.figure(figsize=(7,7))
seaborn.heatmap(train.corr(), annot = True, fmt = '.2f',square = True,vmax=1, vmin = -1,linewidths=0.5, cmap='Dark2')

In [ ]:
cols = ['Deal_value','Weighted_amount','Internal_rating','Success_probability','creation_year','creation_week','creation_day']
i = 1
pyplot.figure(figsize=(20,11))
for col in cols:
    pyplot.subplot(int(str(33)+str(i)))
    seaborn.histplot(data = train, x = col)
    i += 1


In [ ]:
cols = ['Deal_value','Weighted_amount','Internal_rating','Success_probability','creation_year','creation_week','creation_day']
i = 1
pyplot.figure(figsize=(14,11))
for col in cols:
    pyplot.subplot(int(str(33)+str(i)))
    seaborn.violinplot(data = train, y = train[col].astype(float))
    i += 1


# Scaling - Standardscaling

In [ ]:
cols = ['Deal_value','Weighted_amount','Internal_rating','creation_year','creation_week','creation_day']

from sklearn.preprocessing import StandardScaler
train[cols] = StandardScaler().fit_transform(train[cols].values)
valid[cols] = StandardScaler().fit_transform(valid[cols].values)

train

#Imputing Categorical variables

In [ ]:
y = train['Success_probability']
train.drop(columns='Success_probability', inplace= True)

In [ ]:

print(train.shape,valid.shape)
combo = pandas.concat([train,valid])
print(combo.shape)
combo = pandas.get_dummies(combo, drop_first=True)
# ohe = OneHotEncoder(drop='first') 
# ohe.fit_transform(combo)

In [ ]:
combo

In [ ]:
train = combo.iloc[:7007]
valid = combo.iloc[7007:]
train.shape, valid.shape

# PCA

In [ ]:
train.shape,valid.shape

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=0.95, random_state = 42)
pca.fit(train)
pca_train = pca.transform(train)
pca_valid = pca.transform(valid)
train = pandas.DataFrame(data = pca_train)
valid = pandas.DataFrame(data = pca_valid)

# Recursive Feature elimination

In [ ]:
from sklearn.feature_selection import RFECV
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
estimator = DecisionTreeRegressor(criterion='friedman_mse', max_features='auto',
                      max_leaf_nodes=5, min_samples_leaf=28,
                      min_samples_split=5,
                      min_weight_fraction_leaf=0.2777777777777778,
                      random_state=42)
selector = RFECV(estimator, cv=5)
selector = selector.fit(train, y)
selector.support_

In [ ]:
cols = train.columns[selector.support_ == False]
cols
train.drop(columns = cols, inplace=True)
valid.drop(columns = cols, inplace=True)

In [ ]:
# column = pandas.read_csv('rfe.csv')
# column.columns = ['0','1']
# column = column['0'].values
# column

In [ ]:
# train = train[column]
# valid = valid[column]
# train

# Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(train, y, test_size=0.25, random_state=42)

In [ ]:
train.columns.to_series().to_csv('rfe.csv')

# Training Models

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.linear_model import MultiTaskLasso, ElasticNet, MultiTaskElasticNet
from sklearn.linear_model import Lars, LassoLars, OrthogonalMatchingPursuit
from sklearn.linear_model import BayesianRidge, ARDRegression
from sklearn.linear_model import TweedieRegressor, PoissonRegressor,GammaRegressor
from sklearn.linear_model import SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.decomposition import PCA
from sklearn.cross_decomposition import PLSRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVR

from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold
from sklearn.metrics import mean_squared_error, max_error, r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [ ]:

def pcr(n_components=1,whiten = True, svd_solver = 'auto' ):
    return make_pipeline(StandardScaler(), PCA(n_components=n_components,
                                               whiten = whiten,
                                               svd_solver = svd_solver), LinearRegression())
models = [      
    LinearRegression(),
    Ridge(random_state = 42),
    Lasso(random_state = 42),
    ElasticNet(random_state = 42),
    Lars(random_state = 42),
    LassoLars(random_state = 42),
    OrthogonalMatchingPursuit(),
    BayesianRidge(),
    ARDRegression(),
    TweedieRegressor(),
    SGDRegressor(random_state = 42),
    PoissonRegressor(max_iter = 1000),
    GammaRegressor(),
    DecisionTreeRegressor(random_state = 42),
    # RandomForestRegressor(random_state = 42),
    KNeighborsRegressor(),
    SVR(),
    GradientBoostingRegressor(),
    pcr(),
    PLSRegression()
]
params = [
          ['linear', {'normalize' : [True,False]}],
          ['ridge', {'alpha': numpy.logspace(0.01,100,100),
                     'normalize' : [True, False],
                     'solver' : ['auto', 'svd', 'cholesky', 'lsqr', 'sparse_cg', 'sag', 'saga']}],
          ['lasso', {'alpha' : numpy.logspace(0.01, 100, 100),
                     'normalize' : [True, False],
                     'precompute' : ['auto', True, False],
                     'positive' : [True, False],
                     'selection' : ['cyclic', 'random']}],
          ['elasticnet', {'alpha' : numpy.logspace(0.01,100,100),
                          'l1_ratio' : numpy.linspace(0.01, 1, 50),
                          'normalize': [True, False],
                          'selection' : ['cyclic', 'random'],
                          'precompute' : [True, False],
                          'positive' : [True,False]}],
          ['lars', {'normalize': [True, False],
                    'precompute' : [True, False],
                    'eps' : numpy.linspace(0.01, 2, 100),
                    'jitter' : numpy.linspace(0.1, 100, 100)}],
          ['lassolars', {'alpha' : numpy.logspace(0.01,100,100),
                         'normalize': [True, False],
                         'precompute' : [True, False],
                         'positive' :[True, False],
                         'jitter' : numpy.linspace(0.1, 100, 100)}],
          ['omp', {'normalize': [True, False],
                   'precompute' : [True, False]}],
          ['bayesian_ridge', {'alpha_1' : numpy.logspace(0.01, 100,100),
                              'alpha_2' : numpy.logspace(0.01, 100,100),
                              'lambda_1' : numpy.linspace(0.001, 1, 100),
                              'lambda_2' : numpy.linspace(0.001, 1, 100),
                              'lambda_init' : numpy.linspace(0.1, 1, 10),
                              'normalize' : [True, False]}],
          ['ard',  {'alpha_1' : numpy.logspace(0.01, 100,100),
                    'alpha_2' : numpy.logspace(0.01, 100,100),
                    'lambda_1' : numpy.linspace(0.001, 1, 100),
                    'lambda_2' : numpy.linspace(0.001, 1, 100),
                    'threshold_lambda' : numpy.linspace(0.1, 1, 10),
                    'normalize' : [True, False]}],
          ['tweedie', {'power' : [0,1,2,3],
                       'alpha' : numpy.logspace(0.01,100,100),
                       'link' : ['auto', 'identity', 'log']}],
          ['SGD', {'loss' : ['squared_loss', 'huber', 'epsilon_insensitive', 'squared_epsilon_insensitive'],
                   'penalty' : ['l2', 'l1', 'elasticnet'],
                   'alpha' : numpy.logspace(0.01,100,100),
                   'l1_ratio' : numpy.linspace(0.001, 1, 100),
                   'shuffle' : [True,False],
                   'epsilon' : numpy.linspace(0.01, 1,30),
                   'learning_rate' : ['constant', 'invscaling', 'optimal', 'adaptive'],
                   'power_t' : numpy.linspace(0.1, 2, 50),
                   'validation_fraction' : numpy.linspace(0.1, 0.9,10),
                   'average' : [True,False]}],
          ['poisson', {'alpha' : numpy.logspace(0.01,100,100)}],
          ['gamma', {'alpha' : numpy.logspace(0.01,100,100)}],
          ['dtr', {'criterion' : ['mse', 'friedman_mse', 'mae', 'poisson'],
                   'splitter' : ['best', 'random'],
                   'min_samples_split' : range(2, 30),
                   'min_samples_leaf' : range(1, 30),
                   'min_weight_fraction_leaf': numpy.linspace(0,0.5, 10),
                   'max_features' : ['auto', 'sqrt', 'log2'],
                   'max_leaf_nodes' : range(1,30)}],
        #   ['rfr', {'n_estimators' : range(2, 100, 50),
        #            'criterion' : ['mse', 'mae'],
        #            'max_depth' : [None, 10,20,30,40,50,60,70,80,90,100],
        #            'min_samples_split' : range(2, 30),
        #            'min_samples_leaf' : range(1, 30),
        #            'min_weight_fraction_leaf': numpy.linspace(0,0.5, 10),
        #            'max_features' : ['auto', 'sqrt', 'log2'],
        #            'max_leaf_nodes' : range(1,30),
        #            'bootstrap' : [True, False],
        #            'oob_score' : [True,False],
        #            'max_samples' : numpy.linspace(0.1,1, 10),
        #            'ccp_alpha' : numpy.linspace(0.01, 100, 100)}],
          ['knn' , {'n_neighbors' : range(2, 30),
                    'weights' : ['uniform', 'distance'],
                    'algorithm' : ['auto', 'ball_tree', 'kd_tree', 'brute'],
                    'p' : [1,2],
                    'metric' : ['euclidean','manhattan','chebyshev','minkowski','seuclidean','mahalanobis']}],
          ['svr', {}],
          ['gbr', {}],
        #   ['svr', {'kernel' : ['linear', 'poly', 'rbf', 'sigmoid', 'precomputed'],
        #            'degree' : [1,2,3,4,5,6,7,8,9],
        #            'gamma' : ['scale', 'auto'],
        #            'C' : numpy.logspace(1, 100, 100),
        #            'epsilon' : numpy.linspace(0.01, 1, 20),
        #            'shrinking' : [True,False]}],
        #   ['gbr', {'loss' : ['ls', 'lad','huber', 'quantile'],
        #            'learning_rate': numpy.linspace(0.01, 1, 50),
        #            'n_estimators': range(20, 120, 5),
        #            'subsample' : numpy.linspace(0.1, 1.0, 20),
        #            'criterion' : ['friedman_mse', 'mse', 'mae'],
        #            'max_depth' : [None, 10,20,30,40,50,60,70,80,90,100],
        #            'min_samples_split' : range(2, 30),
        #            'min_samples_leaf' : range(1, 30),
        #            'min_weight_fraction_leaf': numpy.linspace(0,0.5, 10),
        #            'ccp_alpha' : numpy.linspace(0.1, 10, 100)}],
          ['pcr', {'pca__n_components' : numpy.linspace(0,1,50),
                   'pca__whiten' : [True, False],
                   'pca__svd_solver' : ['auto', 'full', 'arpack', 'randomized']}],
        #   ['pls', {'n_components' : range(1,1505),
        #            'scale' : [True, False]}],
          ['pls', {}]         
]

In [ ]:
train_score = {'mean_sq_err' : [],
         'max_err': [],
         'r2' : [],
         'percent_err' : []}    

test_score = {'mean_sq_err' : [],
         'max_err': [],
         'r2' : [],
         'percent_err' : []}    

import traceback
for i in range(len(models)):
    try:
        print(params[i][0])
        clf = RandomizedSearchCV(models[i],
                            param_distributions= params[i][1],
                            cv = 5,
                            scoring = "neg_mean_squared_error",
                            n_jobs = 5, verbose = 100)

        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        y_train_pred = clf.predict(x_train)
        train_score['mean_sq_err'].append(clf.best_score_)
        train_score['max_err'].append(max_error(y_train,y_train_pred))
        train_score['r2'].append(r2_score(y_train,y_train_pred))
        train_score['percent_err'].append(mean_absolute_percentage_error(y_train,y_train_pred))

        test_score['mean_sq_err'].append(mean_squared_error(y_test,y_pred))
        test_score['max_err'].append(max_error(y_test,y_pred))
        test_score['r2'].append(r2_score(y_test,y_pred))
        test_score['percent_err'].append(mean_absolute_percentage_error(y_test,y_pred))

        print()
        print('best_estimator :', clf.best_estimator_)
        print('train report :')
        for key in train_score:
            print(f'  {key} : {train_score[key][i]}')

        print('test report :')
        for key in test_score:
            print(f'  {key} : {test_score[key][i]}')
    except Exception as e:
        print('\n\n ', e)
    print("---------------------------------------------------------------------------")

In [ ]:
train_score = pandas.DataFrame(train_score, index = [i[0] for i in params[:]])
test_score = pandas.DataFrame(test_score, index = [i[0] for i in params[:]])
names = [i[0] for i in params]
pyplot.figure(figsize = (25,10))
pyplot.subplot(121)
seaborn.barplot( y = train_score['mean_sq_err'][1:], x = train_score.index[1:])
pyplot.subplot(122)
seaborn.barplot( y = test_score['mean_sq_err'][1:], x = test_score.index[1:])
pyplot.show()
# print(format(train_score['mean_sq_err']))

In [ ]:

clf = RandomizedSearchCV(LassoLars(),
                    param_distributions = {'alpha' : numpy.logspace(0.01,100,100),
                         'normalize': [True, False],
                         'precompute' : [True, False],
                         'positive' :[True, False],
                         'jitter' : numpy.linspace(0.1, 100, 100)},
                    cv = 5,
                    scoring = "neg_mean_squared_error",
                    n_jobs = 5, verbose = 100)

clf.fit(x_train, y_train)
y_pred = clf.predict(x_test)
y_train_pred = clf.predict(x_train)

def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = numpy.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

report(clf.cv_results_)

In [ ]:
train.columns

In [ ]:
# omp best regressor
params = {'precompute': True, 'positive': True, 'normalize': False, 'jitter': 7.163636363636363, 'alpha': 2.187761623949534e+33}
clf = LassoLars(**params)
clf.fit(x_train,y_train)
y_pred = clf.predict(valid)
y_pred.shape, valid.shape

In [ ]:
y_pred

In [ ]:
y_pred = pandas.DataFrame(y_pred, columns = ['Success_probability'])
deal_title = pandas.DataFrame(deal_title, columns = ['Deal_title'])
# # deal_title.reset_index(drop=True, inplace = True)
# # y_pred.reset_index(drop=True, inplace = True)
submit = pandas.concat([deal_title, y_pred], axis = 1 ,ignore_index=True)
# # submit.to_csv('submit.csv')
submit
# deal_title['Success_probability'] = y_pred
# deal_title

In [ ]:
submit.columns = ['Deal_title', 'Success_probability']
submit.to_csv('submit.csv')

#Neural Network Model

In [ ]:
train.columns

In [ ]:
# gridsearch cross validation in neural network model

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.metrics import MeanSquaredError
x_train.astype(int)
x_test.astype(int)
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = numpy.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.3f})"
                  .format(results['mean_test_score'][candidate],
                          results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")

def nn_model(activation = 'relu', neurons = 32, optimizer = 'Adam',dropout = 0.1, init_mode = 'uniform'):
    model = Sequential()
    model.add(Dense(neurons, input_dim = 163, kernel_initializer = init_mode, activation= activation))
    model.add(Dense((neurons*2)//3, kernel_initializer = init_mode,activation= activation))
    model.add(Dense((neurons*4)//9,kernel_initializer = init_mode,  activation = activation))
    model.add(Dropout(dropout))
    model.add(Dense(1, kernel_initializer = init_mode, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer= optimizer, metrics=[MeanSquaredError()])
    return model

# Defining grid parameters
activation = ['softmax', 'softplus', 'softsign', 'relu', 'selu', 'elu', 'tanh','sigmoid', 'linear']
neurons = range(1, 1600, 150)
dropout = [0.0, 0.1, 0.2, 0.3, 0.4, 0.5]
init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
optimizer = ['SGD', 'Adam', 'Adamax','RMSprop','Adagrad','Adadelta','Nadam','Ftrl']
batch_size = range(10,101,10)
param_grid = dict(activation = activation, neurons = neurons, optimizer = optimizer, dropout = dropout, init_mode = init_mode, batch_size = batch_size)

kr = KerasRegressor(build_fn= nn_model, epochs= 5, batch_size = 40, verbose = 1)

model = RandomizedSearchCV(estimator= kr,cv = 3, param_distributions = param_grid, n_jobs=5, verbose = 100)
model.fit(x_train,y_train)

report(model.cv_results_)

In [ ]:
# gridsearch cross validation in neural network model

from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor
from keras.metrics import MeanSquaredError

def nn_model(activation = 'softplus', neurons = 163, optimizer = 'Adamax', dropout = 0.4, init_mode = 'normal'):
    model = Sequential()
    model.add(Dense(neurons, input_dim =163, kernel_initializer = init_mode, activation= activation))
    model.add(Dense((neurons*2)//3, kernel_initializer = init_mode, activation = activation))
    model.add(Dense((neurons*4)//9, kernel_initializer= init_mode,  activation = activation))
    model.add(Dropout(dropout))
    model.add(Dense(1, kernel_initializer = init_mode, activation='linear'))
    model.compile(loss='mean_squared_error', optimizer= optimizer, metrics=[MeanSquaredError()])
    return model

kr = KerasRegressor(build_fn= nn_model, epochs=89, verbose=1, batch_size = 50)
kr.fit(x_train,y_train)
y_train_pred = kr.predict(x_train)
y_pred = kr.predict(x_test)

print('train_score : ')
print('  mean_sq_err : ', mean_squared_error(y_train,y_train_pred))
print('  max_err : ', max_error(y_train,y_train_pred))
print('  r2 : ',r2_score(y_train,y_train_pred))
# print('  percent_err : ', mean_absolute_percentage_error(y_train,y_train_pred))

print('test_score : ') 
print('  mean_sq_err : ', mean_squared_error(y_test,y_pred))
print('  max_err : ', max_error(y_test,y_pred))
print('  r2 : ',r2_score(y_test,y_pred))
# print('  percent_err : ', mean_absolute_percentage_error(y_test,y_pred))


In [ ]:
valid.astype(int)
y_valid = kr.predict(valid)
deal_title = pandas.DataFrame(deal_title)
submit = deal_title.copy()
submit['1'] = y_valid
submit.columns = ['Deal_title', 'Success_probability']
submit.to_csv('submit.csv')

In [ ]:
submit